In [ ]:
#!pip install elasticsearch
#!pip install csv2es
#!pip install requests

In [ ]:
#Comandos Uteis no Terminal
#pip install -U elasticsearch
#sudo -i service elasticsearch status
#sudo -i service elasticsearch start
#sudo -i service elasticsearch restart

In [1]:
from elasticsearch import helpers, Elasticsearch
import csv
import csv2es

In [2]:
es = Elasticsearch()

In [4]:
df = spark.read.csv('DNPBA2017.csv', header=True)

In [5]:
#df = df.select(['NUMERODN', 'CODESTAB', 'IDADEMAE', 'ESCMAEAGR1']).limit(20).toPandas()
df = df.toPandas()

In [6]:
df.to_csv('DNPBA2017_es.csv', index=False)

### indexng

In [4]:
with open('DNPBA2017_es.csv') as f:
    reader = csv.DictReader(f)
    helpers.bulk(es, reader, index='my-index3', doc_type='my-type')
    
# ler indices no linux com: curl 'localhost:9200/_cat/indices?v'

/home/bigdata/anaconda3/lib/python3.8/site-packages/elasticsearch/connection/base.py:193: ElasticsearchDeprecationWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


### buscando termo

In [5]:
content = {
    "query": {
        "term": {
            "CODESTAB": "2786095"
         }
     },
}

In [6]:
res = es.search(index="my-index", body=content)

In [8]:
res

{'took': 1921,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 7, 'relation': 'eq'},
  'max_score': 9.952284,
  'hits': [{'_index': 'my-index',
    '_type': 'my-type',
    '_id': 'wUWOdHcBFOgRnWMjKhSS',
    '_score': 9.952284,
    '_source': {'_c0': '1',
     'NUMERODN': '72390242',
     'CODINST': 'ETO1721000001',
     'ORIGEM': '1',
     'NUMERODV': '0',
     'PREFIXODN': '30',
     'CODESTAB': '2786095',
     'CODMUNNASC': '170700',
     'LOCNASC': '1',
     'IDADEMAE': '25',
     'ESTCIVMAE': '5',
     'ESCMAE': '2',
     'CODOCUPMAE': 'NA',
     'QTDFILVIVO': '04',
     'QTDFILMORT': '00',
     'CODMUNRES': '291110',
     'GESTACAO': '5',
     'GRAVIDEZ': '1',
     'PARTO': '1',
     'CONSULTAS': '2',
     'DTNASC': '24012017',
     'HORANASC': '0245',
     'SEXO': '1',
     'APGAR1': '08',
     'APGAR5': '09',
     'RACACOR': '4',
     'PESO': '3150',
     'IDANOMAL': '2',
     'DTCADASTRO': '14022017',
     

### buscando um intervalo

In [9]:
content = {
    "query": {
        "range": {
            "IDADEMAE": {
                "gte": 20,
                "lt": 30
            }
        }
    }
}

In [10]:
res = es.search(index="my-index", body=content)

In [11]:
res

{'took': 702,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 1.0,
  'hits': [{'_index': 'my-index',
    '_type': 'my-type',
    '_id': 'wUWOdHcBFOgRnWMjKhSS',
    '_score': 1.0,
    '_source': {'_c0': '1',
     'NUMERODN': '72390242',
     'CODINST': 'ETO1721000001',
     'ORIGEM': '1',
     'NUMERODV': '0',
     'PREFIXODN': '30',
     'CODESTAB': '2786095',
     'CODMUNNASC': '170700',
     'LOCNASC': '1',
     'IDADEMAE': '25',
     'ESTCIVMAE': '5',
     'ESCMAE': '2',
     'CODOCUPMAE': 'NA',
     'QTDFILVIVO': '04',
     'QTDFILMORT': '00',
     'CODMUNRES': '291110',
     'GESTACAO': '5',
     'GRAVIDEZ': '1',
     'PARTO': '1',
     'CONSULTAS': '2',
     'DTNASC': '24012017',
     'HORANASC': '0245',
     'SEXO': '1',
     'APGAR1': '08',
     'APGAR5': '09',
     'RACACOR': '4',
     'PESO': '3150',
     'IDANOMAL': '2',
     'DTCADASTRO': '14022017',
     'CODAN

### busca lógica

In [75]:
content = {
    "query": {
        "bool": {
            "must": [
                {
                    "term": {
                        "IDADEMAE": 21
                    }
                },
                {
                    "term": {
                        "ESCMAEAGR1": "06"
                    }
                }
            ]
        }
    }
}

In [76]:
res = es.search(index="my-index", body=content)

In [77]:
res

{u'_shards': {u'failed': 0, u'skipped': 0, u'successful': 1, u'total': 1},
 u'hits': {u'hits': [{u'_id': u'LMatyW8BbundlQk4fUZB',
    u'_index': u'my-index',
    u'_score': 3.6686769,
    u'_source': {u'CODESTAB': u'2777770',
     u'ESCMAEAGR1': u'06',
     u'IDADEMAE': u'21',
     u'NUMERODN': u'69726563'},
    u'_type': u'my-type'}],
  u'max_score': 3.6686769,
  u'total': {u'relation': u'eq', u'value': 1}},
 u'timed_out': False,
 u'took': 5}

### searching methods

In [46]:
def buscaExata(numerodn, codestab, idademae, escmaeagr1, startId=0): 
    
    global es
    
    content = {
        'size': 30,
        'query': {
            'bool': {
                'must': [
                    {'match_phrase': {'NUMERODN': '"' + numerodn + '"'}},
                    {'match_phrase': {'CODESTAB': '"' + codestab + '"'}}, 
                    {'match': {'IDADEMAE': idademae}},
                    {'match': {'ESCMAEAGR1': escmaeagr1}}
                ]
            }
        }
    }
    force = True
    while force:
        try:
            res = es.search(index="my-index", body=content)
            force = False
        except:
            pass
    return res['hits']['hits']

In [52]:
def buscaAproximada(numerodn, codestab, idademae, escmaeagr1, startId=0):
    
    global es
    
    content = {
        'size': 100,
        'query': {
            'bool': {
                'should': [
                    {'match': {'NUMERODN': {'query': numerodn, 'fuzziness':'AUTO', 'operator':'or', 'boost':'2'}}},
                    {'match': {'CODESTAB': {'query': codestab, 'fuzziness':'AUTO', 'operator':'or', 'boost':'2'}}},
                    {'match': {'IDADEMAE': {'query': idademae, 'fuzziness':'AUTO', 'operator':'or', 'boost':'0.5'}}},
                    {'match': {'ESCMAEAGR1': {'query': escmaeagr1, 'fuzziness':'AUTO', 'operator':'or'}}}
                ]
            }
        }
    }
    force = True
    while force:
        try:
            res = es.search(index="my-index", body=content)
            force = False
        except:
            pass
    return res['hits']['hits']

In [53]:
buscaExata("72390242", "2786095", "25", "01")

[{u'_id': u'H8atyW8BbundlQk4fUZA',
  u'_index': u'my-index',
  u'_score': 10.55623,
  u'_source': {u'CODESTAB': u'2786095',
   u'ESCMAEAGR1': u'01',
   u'IDADEMAE': u'25',
   u'NUMERODN': u'72390242'},
  u'_type': u'my-type'}]

In [63]:
# um campo modificado (note que o retorno é vazio)
buscaExata("72390242", "2786095", "33", "01")

[]

In [57]:
buscaAproximada("72390242", "2786095", "25", "01")

[{u'_id': u'H8atyW8BbundlQk4fUZA',
  u'_index': u'my-index',
  u'_score': 9.236701,
  u'_source': {u'CODESTAB': u'2786095',
   u'ESCMAEAGR1': u'01',
   u'IDADEMAE': u'25',
   u'NUMERODN': u'72390242'},
  u'_type': u'my-type'},
 {u'_id': u'I8atyW8BbundlQk4fUZB',
  u'_index': u'my-index',
  u'_score': 5.278115,
  u'_source': {u'CODESTAB': u'7373120',
   u'ESCMAEAGR1': u'12',
   u'IDADEMAE': u'34',
   u'NUMERODN': u'72392109'},
  u'_type': u'my-type'}]

In [65]:
# # um campo modificado (mais de um candidato é retornado)
buscaAproximada("72390242", "2786095", "33", "01")

[{u'_id': u'H8atyW8BbundlQk4fUZA',
  u'_index': u'my-index',
  u'_score': 13.195287,
  u'_source': {u'CODESTAB': u'2786095',
   u'ESCMAEAGR1': u'01',
   u'IDADEMAE': u'25',
   u'NUMERODN': u'72390242'},
  u'_type': u'my-type'},
 {u'_id': u'KMatyW8BbundlQk4fUZB',
  u'_index': u'my-index',
  u'_score': 1.0641159,
  u'_source': {u'CODESTAB': u'2777770',
   u'ESCMAEAGR1': u'08',
   u'IDADEMAE': u'33',
   u'NUMERODN': u'69726416'},
  u'_type': u'my-type'},
 {u'_id': u'KcatyW8BbundlQk4fUZB',
  u'_index': u'my-index',
  u'_score': 1.0641159,
  u'_source': {u'CODESTAB': u'2777770',
   u'ESCMAEAGR1': u'06',
   u'IDADEMAE': u'33',
   u'NUMERODN': u'69726448'},
  u'_type': u'my-type'}]

In [62]:
print len(buscaAproximada("72390242", "2786095", "33", "01"))

3
